In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import gizmo_read

### Prepare file to input into Jeans routine

In [ ]:
sim = "m12m"  # "m12f", "m12i", or "m12m"
part = gizmo_read.read.Read.read_snapshot(species=['star'], 
                                          properties=['position', 'velocity', 'mass', 'massfraction', 'form.scalefactor'], 
                                          directory=f'data/{sim}')


In [ ]:
x = part['star']['position'][:,0]
y = part['star']['position'][:,1]
z = part['star']['position'][:,2]
r = (x**2+y**2+z**2)**0.5

radlim = (r < 600)
r = r[radlim]

$$[M/H] = \log_{10}{\bigg(\frac{Z_{part}}{X_{part}}\bigg)} - \log_{10}{\bigg(\frac{Z_{sun}}{X_{sun}}\bigg)}$$

In [ ]:
Y_part = part['star']['massfraction'][:,1][radlim]  # mass fraction of He
Z_part = part['star']['massfraction'][:,0][radlim]  # mass fraction of metals (everything except H, He)
X_part = 1 - (Y_part + Z_part)                      # mass fraction of H
print(f"X_part_avg={np.mean(X_part):.4f}, Y_part_avg={np.mean(Y_part):.4f}, Z_part_avg={np.mean(Z_part):.4f}")

X_sun = gizmo_read.constant.sun_composition['hydrogen']['massfraction']
Y_sun = gizmo_read.constant.sun_composition['helium']['massfraction']
Z_sun = gizmo_read.constant.sun_composition['metals']['massfraction']
print(f"X_sun={X_sun:.4f}, Y_sun={Y_sun:.4f}, Z_sun={Z_sun:.4f}")

# [M/H]
metallicity = np.log10(Z_part/X_part) - np.log10(Z_sun/X_sun)

plt.figure()
plt.ylabel('[M/H]')
plt.xlabel('Radius (kpc)')
plt.ylim([-5.5,2])
plt.scatter(r[::100], metallicity[::100], marker='.', alpha=0.1)
plt.axhline(-1.5, c='r', label='[M/H] = -1.5 cutoff')
plt.legend()
plt.show()

In [ ]:
age = part['star']['age'][radlim]

plt.figure()
plt.ylabel('Age (Gyr)')
plt.xlabel('Radius (kpc)')
plt.scatter(r[::100], age[::100], marker='.', alpha=0.1)
plt.axhline(8, c='r', label='Age = 8 Gyr cutoff')
plt.legend()
plt.show()

In [ ]:
# [M/H] and age threshold to select halo stars
halo = (metallicity < -1.5) * (age > 8)

In [ ]:
# apply halo selection to all quantities
x = x[radlim][halo]; y = y[radlim][halo]; z = z[radlim][halo]; r = r[halo]
vx = part['star']['velocity'][:,0][radlim][halo]
vy = part['star']['velocity'][:,1][radlim][halo]
vz = part['star']['velocity'][:,2][radlim][halo]
m  = part['star']['mass'][radlim][halo]

In [ ]:
# precompute spherical velocities for use in jeans
sphvels = gizmo_read.coordinate.get_velocities_in_coordinate_system(part['star']['velocity'][radlim][halo], 
                                                                    part['star']['position'][radlim][halo],
                                                                    system_from='cartesian',
                                                                    system_to='spherical')
vr, vtheta, vphi = np.transpose(sphvels)

In [ ]:
# Write data to disk
np.savetxt(
    fname=f"data/{sim}_prejeans.csv", 
    X=np.stack([x, y, z, vx, vy, vz, m, r, vr**2, vtheta**2, vphi**2], axis=1), delimiter=',', 
    header="x, y, z [kpc], vx, vy, vz [km/s], mass [Msun], gc_radius, vr_sq, vtheta_sq, vphi_sq [km2/s2]"
)

### Prepare file to plot true M(<r) profile

In [ ]:
# Need radii and masses of all particle types for true enclosed mass profile
sim = "m12f"
part = gizmo_read.read.Read.read_snapshot(species=['star', 'dark', 'gas'], 
                                          properties=['position', 'mass'], 
                                          directory=f'data/{sim}')


In [ ]:
# calculate radii of each particle type
r_star = (part['star']['position'][:,0]**2+part['star']['position'][:,1]**2+part['star']['position'][:,2]**2)**0.5
r_dark = (part['dark']['position'][:,0]**2+part['dark']['position'][:,1]**2+part['dark']['position'][:,2]**2)**0.5
r_gas  = (part['gas'] ['position'][:,0]**2+part['gas'] ['position'][:,1]**2+part['gas'] ['position'][:,2]**2)**0.5

radlim_star = (r_star < 600); radlim_dark = (r_dark < 600); radlim_gas  = (r_gas < 600)

r_star = r_star[radlim_star]; r_dark = r_dark[radlim_dark]; r_gas = r_gas[radlim_gas]

In [ ]:
# collect radii and masses of all particles and sort by radius
all_radii = np.concatenate((r_star, r_dark, r_gas))
all_masses = np.concatenate((part['star']['mass'][radlim_star], 
                             part['dark']['mass'][radlim_dark], 
                             part['gas']['mass'][radlim_gas]))

sorter = np.argsort(all_radii)
all_radii  = all_radii [sorter]
all_masses = all_masses[sorter]

# cumulative sum of sorted masses is mass enclosed
Menc = np.cumsum(all_masses, dtype=np.float64)

In [ ]:
# Thin profile to save storage space but maintain enough resolution for a smooth profile
plt.plot(all_radii[::500], Menc[::500])
plt.xlim([0,100])
plt.xlabel('gc radius [kpc]')
plt.ylabel('True M(<r) [Msun]')
plt.show()

In [ ]:
# Write data to disk
np.savetxt(
    fname=f"data/{sim}_true.csv", 
    X=np.stack([all_radii[::500], Menc[::500]], axis=1), delimiter=',', 
    header="r [kpc], M(<r)_true [Msun]"
)

In [ ]:
# clear up some memory after writing file
del part, r_star, r_dark, r_gas, radlim_star, radlim_dark, radlim_gas, all_radii, all_masses, sorter, Menc